# 데이터 로드 및 전처리

In [ ]:
!pip install xgboost
!pip install lightgbm
!pip install catboost

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,mean_absolute_percentage_error
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib

#한글꺠짐 방지
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

In [2]:
data=pd.read_csv("./data/class_balance.csv",encoding="EUC-KR")
data['Y'] = data['Y'].map(lambda x: 0 if x == -1 else 1)
#X,Y 분할
Y=data["Y"].copy()
X=data.drop("Y",axis=1)
X.head(3)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X465,X466,X467,X468,X469,X470,X471,X472,X473,X474
0,0.457896,0.530189,0.276976,0.359864,0.193059,0.322190,0.706897,0.553781,0.653894,0.375204,...,0.283280,0.246376,0.711806,0.008532,0.013672,0.008467,0.402240,0.238811,0.274876,0.210238
1,0.607100,0.341478,0.518992,0.395300,0.042071,0.469654,0.750000,0.542031,0.447466,0.189233,...,0.328358,0.523785,0.760417,0.030930,0.033203,0.029759,0.210356,0.309339,0.328358,0.439175
2,0.360781,0.369653,0.341039,0.021697,0.181737,0.528684,0.491379,0.516722,0.300371,0.376835,...,0.208955,0.185769,0.659722,0.005333,0.003906,0.005311,0.381877,0.208171,0.208955,0.155761


In [3]:
data

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X466,X467,X468,X469,X470,X471,X472,X473,X474,Y
0,0.457896,0.530189,0.276976,0.359864,0.193059,0.322190,0.706897,0.553781,0.653894,0.375204,...,0.246376,0.711806,0.008532,0.013672,0.008467,0.402240,0.238811,0.274876,0.210238,0
1,0.607100,0.341478,0.518992,0.395300,0.042071,0.469654,0.750000,0.542031,0.447466,0.189233,...,0.523785,0.760417,0.030930,0.033203,0.029759,0.210356,0.309339,0.328358,0.439175,0
2,0.360781,0.369653,0.341039,0.021697,0.181737,0.528684,0.491379,0.516722,0.300371,0.376835,...,0.185769,0.659722,0.005333,0.003906,0.005311,0.381877,0.208171,0.208955,0.155761,0
3,0.460910,0.413500,0.535685,0.302794,0.242326,0.408966,0.646552,0.561615,0.415328,0.313214,...,0.246376,0.000000,1.000000,1.000000,1.000000,0.381877,0.208171,0.208955,0.155761,0
4,0.263068,0.279821,0.535685,0.302794,0.242326,0.408966,0.646552,0.638747,0.660074,0.520392,...,0.110711,0.517361,0.023677,0.022461,0.023447,0.608414,0.212062,0.268657,0.092827,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,0.360022,0.396033,0.382803,0.070771,0.143308,0.920884,0.586207,0.636336,0.337454,0.432300,...,0.000000,0.597222,0.016212,0.013672,0.015997,0.045307,0.147860,0.171642,0.000000,1
250,0.508628,0.437628,0.192378,0.061866,0.168425,0.481919,0.715517,0.270563,0.407911,0.336052,...,0.132582,0.975694,0.017065,0.015625,0.016114,0.543689,0.227626,0.320896,0.111165,1
251,0.275930,0.364365,0.302236,0.376615,0.485135,0.627270,0.594828,0.435673,0.420272,0.367047,...,0.246376,0.586806,0.008532,0.015625,0.008613,0.343042,0.151751,0.164179,0.143012,1
252,0.785179,0.271804,0.400189,0.457851,0.229526,0.244320,0.396552,0.875565,0.110012,0.337684,...,0.221768,0.767361,0.024317,0.019531,0.023461,0.509709,0.398833,0.440299,0.185945,1


# 평가 지표 함수

In [4]:
# 평가 지표 출력 함수
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix

def print_all_reg(Y_test,pred):
    # Specificity를 구하기 위해 confusion matrix를 이용
    cm1 = confusion_matrix(Y_test,pred)
    specificity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    
    #결과 검사
    #recall = cm1[1,1]/(cm1[1,1]+cm1[1,0])
    #pre = cm1[1,1]/(cm1[1,1]+cm1[0,1])

    G_mean = recall_score(Y_test,pred) * specificity1
    
    print("model의 recall 값은 {:.3f}".format(recall_score(Y_test,pred)))
    print("model의 2종 오류 확률 값은 {:.3f}".format(1-recall_score(Y_test,pred)))
    print("model의 Specificity 값은 {:.3f}".format(specificity1))
    print("model의 1종 오류 확률 값은 {:.3f}".format(1-specificity1))
    print("model의 precision 값은 {:.3f}".format(precision_score(Y_test,pred)))
    print("model의 f1_score 값은 {:.3f}".format(f1_score(Y_test,pred)))
    print("model의 G-mean 값은 {:.3f}".format(np.sqrt(G_mean)))
    print("model의 accuracy 값은 {:.3f}".format(accuracy_score(Y_test,pred)))


In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=5,shuffle =True)

# 모델 학습 및 예측

In [6]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import math
import time
import datetime


def evaluate_regression_models(X, y, model_name):
    # 데이터 생성    
    # 모델 학습
    if model_name == "GBM":
        # Gradient Boosting 모델
        X_train, X_eval, y_train, Y_eval = train_test_split(X, y, test_size=0.2, random_state=42)
        model = GradientBoostingClassifier(max_depth = 8,n_estimators=500,validation_fraction = 0.2, n_iter_no_change = 10,learning_rate = 0.005)
        model.fit(X_train, y_train)
        print("학습 데이터 성능")
        preds = model.predict(X_train)
        print_all_reg(preds,y_train)

    elif model_name == "XGB":        
        # XGBoost 모델
        X_train, X_eval, y_train, Y_eval = train_test_split(X, y, test_size=0.2, random_state=42)
        eval_set = [(X_eval, Y_eval)]
        model = xgb.XGBClassifier(n_estimators=200,learning_rate = 0.005,
                        min_child_weight = 2,
                        colsample_bytree = 0.7,
                        max_depth = 8)
        model.fit(X_train, y_train,eval_set = eval_set,early_stopping_rounds = 10)
        
        print("학습 데이터 성능")
        preds = model.predict(X_train)
        print_all_reg(preds,y_train)
    
    elif model_name == "Light":     
        params = {'learning_rate': 0.01, 
          'max_depth': 16, 
          'boosting': 'gbdt', 
          'metric': 'accuracy', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':2018}

#         # LightGBM 모델
#         X_train, X_eval, y_train, Y_eval = train_test_split(X, y, test_size=0.2, random_state=42)
#         train_ds = lgb.Dataset(X_train, label = y_train) 
#         test_ds = lgb.Dataset(X_eval, label = Y_eval) 
#         model = lgb.train(params, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=10)
        X_train, X_eval, y_train, Y_eval = train_test_split(X, y, test_size=0.2, random_state=42)
        eval_set = [(X_eval, Y_eval)]
        model = lgb.LGBMClassifier(max_depth = 8,min_child_weight = 2,n_estimators=200,learning_rate = 0.009)
        model.fit(X_train, y_train,eval_set = eval_set,early_stopping_rounds = 5,eval_metric='accuracy')
        print("학습 데이터 성능")
        preds = model.predict(X_train)
        print_all_reg(preds,y_train)

    elif model_name == "Cat":     
        # CatBoost 모델
        X_train, X_eval, y_train, Y_eval = train_test_split(X, y, test_size=0.2, random_state=42)
        eval_set = [(X_eval, Y_eval)]
        model = cb.CatBoostClassifier(max_depth = 6,n_estimators=200,logging_level='Silent',learning_rate = 0.001)
        model.fit(X_train, y_train,eval_set = eval_set,early_stopping_rounds = 5)
        print("학습 데이터 성능")
        preds = model.predict(X_train)
        print_all_reg(preds,y_train)

    

    return model

# GBM 성능 평가

In [7]:
print("테스트 데이터 성능")
start_time = time.time()
model= evaluate_regression_models(X_train, Y_train,"GBM")
print("Time: {:.4f}sec".format((time.time() - start_time)))
preds = model.predict(X_test)
print_all_reg(preds,Y_test)

테스트 데이터 성능
학습 데이터 성능
model의 recall 값은 0.924
model의 2종 오류 확률 값은 0.076
model의 Specificity 값은 0.931
model의 1종 오류 확률 값은 0.069
model의 precision 값은 0.884
model의 f1_score 값은 0.904
model의 G-mean 값은 0.928
model의 accuracy 값은 0.929
Time: 0.6967sec
model의 recall 값은 1.000
model의 2종 오류 확률 값은 0.000
model의 Specificity 값은 0.545
model의 1종 오류 확률 값은 0.455
model의 precision 값은 0.286
model의 f1_score 값은 0.444
model의 G-mean 값은 0.739
model의 accuracy 값은 0.615


# XGBoost 성능 평가

In [8]:
start_time = time.time()
model= evaluate_regression_models(X_train, Y_train,"XGB")
print("Time: {:.4f}sec".format((time.time() - start_time)))
preds = model.predict(X_test)
print_all_reg(preds,Y_test)

[0]	validation_0-logloss:0.69054
[1]	validation_0-logloss:0.68831
[2]	validation_0-logloss:0.68583
[3]	validation_0-logloss:0.68394
[4]	validation_0-logloss:0.68219
[5]	validation_0-logloss:0.68055
[6]	validation_0-logloss:0.67876
[7]	validation_0-logloss:0.67641
[8]	validation_0-logloss:0.67486
[9]	validation_0-logloss:0.67339
[10]	validation_0-logloss:0.67152
[11]	validation_0-logloss:0.67003
[12]	validation_0-logloss:0.66841
[13]	validation_0-logloss:0.66670
[14]	validation_0-logloss:0.66529
[15]	validation_0-logloss:0.66383
[16]	validation_0-logloss:0.66237
[17]	validation_0-logloss:0.66080
[18]	validation_0-logloss:0.65933
[19]	validation_0-logloss:0.65787
[20]	validation_0-logloss:0.65576
[21]	validation_0-logloss:0.65415
[22]	validation_0-logloss:0.65252
[23]	validation_0-logloss:0.65112
[24]	validation_0-logloss:0.65003
[25]	validation_0-logloss:0.64823
[26]	validation_0-logloss:0.64651
[27]	validation_0-logloss:0.64529
[28]	validation_0-logloss:0.64394
[29]	validation_0-loglos

C:\Users\hyeonggu\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[41]	validation_0-logloss:0.62816
[42]	validation_0-logloss:0.62700
[43]	validation_0-logloss:0.62583
[44]	validation_0-logloss:0.62453
[45]	validation_0-logloss:0.62337
[46]	validation_0-logloss:0.62190
[47]	validation_0-logloss:0.62101
[48]	validation_0-logloss:0.61997
[49]	validation_0-logloss:0.61891
[50]	validation_0-logloss:0.61766
[51]	validation_0-logloss:0.61657
[52]	validation_0-logloss:0.61560
[53]	validation_0-logloss:0.61455
[54]	validation_0-logloss:0.61310
[55]	validation_0-logloss:0.61179
[56]	validation_0-logloss:0.61082
[57]	validation_0-logloss:0.60933
[58]	validation_0-logloss:0.60837
[59]	validation_0-logloss:0.60711
[60]	validation_0-logloss:0.60596
[61]	validation_0-logloss:0.60517
[62]	validation_0-logloss:0.60412
[63]	validation_0-logloss:0.60333
[64]	validation_0-logloss:0.60236
[65]	validation_0-logloss:0.60112
[66]	validation_0-logloss:0.60024
[67]	validation_0-logloss:0.59931
[68]	validation_0-logloss:0.59850
[69]	validation_0-logloss:0.59756
[70]	validatio

# LightGBM

In [9]:
start_time = time.time()
model= evaluate_regression_models(X_train, Y_train,"Light")
print("Time: {:.4f}sec".format((time.time() - start_time)))
preds = model.predict(X_test)
print_all_reg(preds,Y_test)

[1]	valid_0's binary_logloss: 0.697985
[2]	valid_0's binary_logloss: 0.694067
[3]	valid_0's binary_logloss: 0.690752
[4]	valid_0's binary_logloss: 0.686988
[5]	valid_0's binary_logloss: 0.683463
[6]	valid_0's binary_logloss: 0.680351
[7]	valid_0's binary_logloss: 0.676799
[8]	valid_0's binary_logloss: 0.673348
[9]	valid_0's binary_logloss: 0.670419
[10]	valid_0's binary_logloss: 0.667157
[11]	valid_0's binary_logloss: 0.663488
[12]	valid_0's binary_logloss: 0.659556
[13]	valid_0's binary_logloss: 0.656817
[14]	valid_0's binary_logloss: 0.653019
[15]	valid_0's binary_logloss: 0.649885
[16]	valid_0's binary_logloss: 0.646195
[17]	valid_0's binary_logloss: 0.643256
[18]	valid_0's binary_logloss: 0.640727
[19]	valid_0's binary_logloss: 0.637414
[20]	valid_0's binary_logloss: 0.634103
[21]	valid_0's binary_logloss: 0.631591
[22]	valid_0's binary_logloss: 0.628625
[23]	valid_0's binary_logloss: 0.62432
[24]	valid_0's binary_logloss: 0.621997
[25]	valid_0's binary_logloss: 0.619212
[26]	valid

C:\Users\hyeonggu\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


# Catboost

In [95]:
start_time = time.time()
model= evaluate_regression_models(X_train, Y_train,"Cat")
print("Time: {:.4f}sec".format((time.time() - start_time)))
preds = model.predict(X_test)
print_all_reg(preds,Y_test)

학습 데이터 성능
model의 recall 값은 1.000
model의 2종 오류 확률 값은 0.000
model의 Specificity 값은 0.919
model의 1종 오류 확률 값은 0.081
model의 precision 값은 0.855
model의 f1_score 값은 0.922
model의 G-mean 값은 0.958
model의 accuracy 값은 0.945
Time: 1.7086sec
model의 recall 값은 1.000
model의 2종 오류 확률 값은 0.000
model의 Specificity 값은 0.667
model의 1종 오류 확률 값은 0.333
model의 precision 값은 0.571
model의 f1_score 값은 0.727
model의 G-mean 값은 0.816
model의 accuracy 값은 0.769
